# 🚀 Projet TortillaPredict !

**Ce projet aura pour objectif d'étudier d'étudier des données relatives aux prix de vente de tortillas, au mexique, en fonction de différentes caractéristiques.**

Nous procéderons dans un premier temps à une analyse des données afin de mieux les appréhender, puis nous tenterons d'établir un modèle de prédiction permettant de prédire le prix auquel sera vendue une tortilla en fonction de ses caractéristiques.

In [1]:
#Chargement des librairies nécessaires pour le traitement des données
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
#Chargement des données
tortilla_datas = pd.read_csv('tortilla_prices.csv')

In [10]:
#Commençons par observer les premières lignes de notre jeu de données
print(tortilla_datas.head(20))

                  State              City  Year  Month  Day  \
0        Aguascalientes    Aguascalientes  2007      1   10   
1       Baja California          Mexicali  2007      1   10   
2       Baja California           Tijuana  2007      1   10   
3   Baja California Sur            La Paz  2007      1   10   
4              Campeche          Campeche  2007      1   10   
5              Coahuila    Piedras Negras  2007      1   10   
6              Coahuila          Saltillo  2007      1   10   
7                Colima            Colima  2007      1   10   
8               Chiapas         Tapachula  2007      1   10   
9               Chiapas  Tuxtla Gutiérrez  2007      1   10   
10            Chihuahua         Chihuahua  2007      1   10   
11            Chihuahua        Cd. Juárez  2007      1   10   
12                 D.F.              D.F.  2007      1   10   
13                 D.F.           ZM D.F.  2007      1   10   
14              Durango           Durango  2007      1 

TODO: Observation sur la premère impression des données

In [11]:
#Chopons les premières infos sur notre jeu de données

print(tortilla_datas.info())
print("-------------------------------------------------------------------------------------------------------\n")
print(tortilla_datas.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278886 entries, 0 to 278885
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   State               278886 non-null  object 
 1   City                278886 non-null  object 
 2   Year                278886 non-null  int64  
 3   Month               278886 non-null  int64  
 4   Day                 278886 non-null  int64  
 5   Store type          278886 non-null  object 
 6   Price per kilogram  272496 non-null  float64
dtypes: float64(1), int64(3), object(3)
memory usage: 14.9+ MB
None
-------------------------------------------------------------------------------------------------------

                Year          Month            Day  Price per kilogram
count  278886.000000  278886.000000  278886.000000       272496.000000
mean     2015.080015       6.384781      15.951955           12.054586
std         4.912392       3.440216       8.755248    

In [12]:
#On va verifier la qualité du dataset
#Y a t il des valeurs manquantes?

nan_values = tortilla_datas.isna().sum()

print(nan_values)

State                    0
City                     0
Year                     0
Month                    0
Day                      0
Store type               0
Price per kilogram    6390
dtype: int64


On constate que dans la colonne des prix au kg on a pas mal de valeur nulles, pour poursuivre l'analyse, on va virer les valeurs manquantes.

In [16]:
clean_tortilla_datas = tortilla_datas.dropna()

print(clean_tortilla_datas.info())

print("\n--------------------------------------------------------------------------------------------------------\n")

print(clean_tortilla_datas.isna().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 272496 entries, 0 to 278885
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   State               272496 non-null  object 
 1   City                272496 non-null  object 
 2   Year                272496 non-null  int64  
 3   Month               272496 non-null  int64  
 4   Day                 272496 non-null  int64  
 5   Store type          272496 non-null  object 
 6   Price per kilogram  272496 non-null  float64
dtypes: float64(1), int64(3), object(3)
memory usage: 16.6+ MB
None

--------------------------------------------------------------------------------------------------------

State                 0
City                  0
Year                  0
Month                 0
Day                   0
Store type            0
Price per kilogram    0
dtype: int64
